In [66]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97.5% !important; }</style>"))

In [67]:
%reload_ext autoreload
%autoreload 2
import numpy as np
from dops import *

In [68]:
def generate_data(u, theta, C, n, k):
    S = np.zeros((n, len(u)))
    z = np.zeros(n)
    for i in range(n):
        features = np.random.choice(u, size=k, replace=False)
        S[i] = np.array([1 if j in features else 0 for j in range(len(u))])
        cs = []
        for f in features:
            for j in C[f]:
                if j not in cs:
                    cs.append(j)
        z[i] = np.sum(theta[cs])
    return S, z

k = 5
d = 20
u = np.arange(d)
theta = np.random.randint(100, size=d)
C = [np.random.choice(u, size=np.random.randint(1,8), replace=False) for _ in range(d)]
S, z = generate_data(u, theta, C, 90, k)
T, v = generate_data(u, theta, C, 10, k)

In [71]:
C

[array([19, 12,  4, 13,  1, 14]),
 array([5]),
 array([12, 14, 18,  9,  7]),
 array([7]),
 array([ 8,  3, 16, 14,  7]),
 array([ 1, 14]),
 array([12,  9, 11, 16,  4,  7, 10]),
 array([16]),
 array([9, 0]),
 array([17, 16,  3,  7]),
 array([9, 3, 2, 7]),
 array([11]),
 array([ 9,  0, 18, 19, 16,  1]),
 array([15, 13]),
 array([16,  1, 15]),
 array([8, 6]),
 array([14,  6]),
 array([10, 12]),
 array([14, 11, 16, 12,  3,  0]),
 array([18, 17,  1, 12])]

In [59]:
indices_of_results = []
for i in range(20):
    k = 5
    d = 20
    u = np.arange(d)
    theta = np.random.randint(100, size=d)
    C = [np.random.choice(u, size=np.random.randint(1,8), replace=False) for _ in range(d)]
    S, z = generate_data(u, theta, C, 90, k)
    T, v = generate_data(u, theta, C, 10, k)
    res, pred_theta, max_item_index = dops(S, z, T, C, 6, 0.5, np.zeros(d), eta=0.02, iters=1500, verbose=False)
    max_item = v[max_item_index]
    v = sorted(v,reverse=True)
    indices_of_results.append(v.index(max_item))

In [63]:
##Expected the indices to be small 1 ~ 3 but instead the results are relatively high
##When we change the verbose to true, we find that our theta doesn't converge, which is unexpected.
indices_of_results 

[9, 1, 5, 8, 6, 8, 9, 5, 5, 4, 5, 6, 4, 0, 4, 1, 7, 9, 8, 1]

In [36]:
pred_theta, theta

(array([ 5.33333333e-02,  6.00000000e-02, -9.90527105e-15,  2.66666667e-02,
        -1.13333333e-01, -1.33333333e-02, -6.66666667e-03,  2.66666667e-02,
         3.33333333e-02, -3.33333333e-02, -8.19136425e-15,  6.66666667e-03,
        -2.66666667e-02,  6.66666667e-03,  4.66666667e-02, -2.00000000e-02,
         1.33333333e-02,  3.33333333e-02, -2.00000000e-02, -8.66666667e-02]),
 array([37, 10, 83, 13, 26, 82, 35, 47, 77, 93, 28, 64, 67, 44, 35, 61, 46,
         6, 51, 77]))

In [37]:
res

[0.16666666666659724,
 -0.1066666666667547,
 0.06666666666660068,
 0.14666666666659703,
 0.09999999999994141,
 0.09999999999991441,
 -0.10000000000008119,
 0.026666666666588665,
 0.11333333333325114,
 0.07999999999992463]